In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Rebuilding the Context Pipeline for an AI Legal Research Assistant
## Implementation Notebook

Welcome to the implementation notebook for the BriefEngine context engineering case study. In this notebook, you will build a complete context engineering pipeline for a legal research assistant, progressing from data exploration through a fully evaluated retrieval-augmented generation system.

**What you will build:**
- A filtered retrieval pipeline over real U.S. case law data
- A semantic search and reranking system using sentence transformers
- A context assembly engine that implements all four context engineering strategies (Write, Select, Compress, Isolate)
- A citation verification system
- A complete evaluation framework comparing your pipeline against a BM25 baseline

**Prerequisites:** Familiarity with Python, NumPy, and basic NLP concepts. Understanding of the context engineering article (the four failure modes and four strategies) is essential.

**Estimated time:** 3-4 hours

---

## 3.1 Data Acquisition and Preprocessing

In this section, you will load and preprocess a subset of the CaseLaw Access Project (Harvard Law School) — a corpus of real U.S. court decisions. We work with California state courts and Ninth Circuit federal opinions from 2000-2024.

The dataset has been pre-processed into JSON records. Each record contains the full opinion text, court metadata, date, citation string, jurisdiction, and a boolean indicating whether the case has been overruled by a later decision.

In [ ]:
# Install dependencies
!pip install -q sentence-transformers rank_bm25 numpy pandas matplotlib seaborn tqdm

import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

print("All dependencies loaded successfully.")

In [ ]:
# --- Synthetic Dataset Generation ---
# Since the full CaseLaw Access Project requires bulk download approval,
# we generate a realistic synthetic dataset that mirrors its structure.
# The synthetic data preserves realistic: court names, citation formats,
# opinion lengths, jurisdiction distributions, and citation networks.

import random
import hashlib

random.seed(42)
np.random.seed(42)

# California courts and Ninth Circuit federal courts
COURTS = {
    "CA_supreme": "Supreme Court of California",
    "CA_appeal_1": "California Court of Appeal, First District",
    "CA_appeal_2": "California Court of Appeal, Second District",
    "CA_appeal_4": "California Court of Appeal, Fourth District",
    "9th_circuit": "United States Court of Appeals for the Ninth Circuit",
    "ND_CA": "United States District Court for the Northern District of California",
    "CD_CA": "United States District Court for the Central District of California",
}

JURISDICTIONS = {
    "CA_supreme": "california",
    "CA_appeal_1": "california",
    "CA_appeal_2": "california",
    "CA_appeal_4": "california",
    "9th_circuit": "federal-9th",
    "ND_CA": "federal-9th",
    "CD_CA": "federal-9th",
}

LEGAL_TOPICS = [
    "negligence", "strict liability", "breach of contract", "due process",
    "equal protection", "employment discrimination", "wrongful termination",
    "product liability", "medical malpractice", "premises liability",
    "insurance bad faith", "fraud", "unfair business practices",
    "intellectual property", "trade secrets", "environmental law",
    "landlord-tenant", "family law", "criminal procedure", "civil rights",
    "first amendment", "fourth amendment", "habeas corpus",
    "class action", "arbitration", "statute of limitations",
    "emotional distress", "defamation", "privacy", "antitrust",
]

LEGAL_PHRASES = [
    "The court finds that the defendant's conduct constitutes",
    "Under California Civil Code section",
    "The standard of review for this claim is",
    "Applying the test established in",
    "The plaintiff has demonstrated a prima facie case of",
    "The doctrine of res judicata bars this claim because",
    "Pursuant to the holding in",
    "The court grants summary judgment on the grounds that",
    "The evidence presented at trial establishes that",
    "Under the substantial evidence standard, we conclude",
    "The trial court did not abuse its discretion in",
    "We review de novo the question of whether",
    "The statute requires a showing of",
    "The burden of proof shifts to the defendant to demonstrate",
    "Considering the totality of the circumstances,",
]


def generate_opinion_text(topic: str, length_tokens: int) -> str:
    """Generate a synthetic legal opinion on a given topic."""
    paragraphs = []
    tokens_so_far = 0
    while tokens_so_far < length_tokens:
        phrase = random.choice(LEGAL_PHRASES)
        detail = (
            f"{topic}. The court examined the relevant statutory provisions "
            f"and prior case law to determine the applicable standard. "
            f"After careful consideration of the evidence and arguments "
            f"presented by both parties, the court concludes that the "
            f"requirements have been {'satisfied' if random.random() > 0.4 else 'not met'}. "
            f"This determination is consistent with the precedent established "
            f"in prior decisions of this court and the {'California Supreme Court' if random.random() > 0.5 else 'Ninth Circuit'}."
        )
        paragraph = f"{phrase} {detail}"
        paragraphs.append(paragraph)
        tokens_so_far += len(paragraph.split())
    return "\n\n".join(paragraphs)


def generate_citation(court_key: str, year: int, idx: int) -> str:
    """Generate a realistic citation string."""
    if court_key.startswith("CA_supreme"):
        return f"{random.randint(1, 60)} Cal.{random.choice(['3d', '4th', '5th'])} {random.randint(100, 999)} ({year})"
    elif court_key.startswith("CA_appeal"):
        return f"{random.randint(1, 300)} Cal.App.{random.choice(['4th', '5th'])} {random.randint(100, 999)} ({year})"
    elif court_key == "9th_circuit":
        return f"{random.randint(100, 999)} F.3d {random.randint(100, 1500)} ({year})"
    else:
        return f"{random.randint(100, 999)} F.Supp.{random.choice(['2d', '3d'])} {random.randint(100, 1500)} ({year})"


def generate_case_name() -> str:
    """Generate a realistic case name."""
    first_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia",
                   "Miller", "Davis", "Rodriguez", "Martinez", "Hernandez",
                   "Lopez", "Gonzalez", "Wilson", "Anderson", "Thomas"]
    entities = ["Pacific Mutual", "State of California", "City of Los Angeles",
                "County of San Francisco", "United Healthcare", "Tesla Motors",
                "Pacific Gas & Electric", "Wells Fargo", "Kaiser Permanente",
                "University of California", "Walmart Inc.", "Amazon.com LLC",
                "Department of Motor Vehicles", "Board of Education"]

    plaintiff = random.choice(first_names + entities)
    defendant = random.choice(first_names + entities)
    while defendant == plaintiff:
        defendant = random.choice(first_names + entities)
    return f"{plaintiff} v. {defendant}"


# Generate 2,000 synthetic cases (scaled for Colab)
NUM_CASES = 2000
cases = []

for i in range(NUM_CASES):
    court_key = random.choice(list(COURTS.keys()))
    year = random.randint(2000, 2024)
    topic = random.choice(LEGAL_TOPICS)
    length = random.randint(200, 2000)
    is_overruled = random.random() < 0.05  # 5% overruled rate

    case = {
        "id": f"case_{i:05d}",
        "case_name": generate_case_name(),
        "court": COURTS[court_key],
        "court_key": court_key,
        "jurisdiction": JURISDICTIONS[court_key],
        "year": year,
        "citation": generate_citation(court_key, year, i),
        "topic": topic,
        "is_overruled": is_overruled,
        "overruled_by": generate_citation(court_key, year + random.randint(1, 5), i + NUM_CASES) if is_overruled else None,
        "text": generate_opinion_text(topic, length),
    }
    cases.append(case)

df_cases = pd.DataFrame(cases)
print(f"Generated {len(cases)} synthetic cases")
print(f"Courts: {df_cases['court'].nunique()} unique courts")
print(f"Years: {df_cases['year'].min()} - {df_cases['year'].max()}")
print(f"Overruled: {df_cases['is_overruled'].sum()} cases ({df_cases['is_overruled'].mean()*100:.1f}%)")
print(f"\nSample case:")
print(json.dumps({k: v for k, v in cases[0].items() if k != 'text'}, indent=2))
print(f"Text preview: {cases[0]['text'][:200]}...")

In [ ]:
# --- Chunking ---
# Legal opinions are too long to embed as single documents.
# We split them into overlapping chunks that preserve context.

def chunk_text(text: str, chunk_size: int = 512, overlap: int = 64) -> List[str]:
    """Split text into overlapping token-level chunks.

    Args:
        text: The full opinion text
        chunk_size: Maximum tokens per chunk (using word-level approximation)
        overlap: Number of overlapping tokens between consecutive chunks

    Returns:
        List of text chunks

    Hints:
        1. Split text into words (tokens ~ words for this approximation)
        2. Use a sliding window with step = chunk_size - overlap
        3. Join words back into strings for each chunk
        4. Handle the final chunk (may be shorter than chunk_size)
        5. Ensure no empty chunks are returned
    """
    # TODO: Implement this function
    pass


# Verification cell
# After implementing chunk_text, run this to verify:
sample_text = "word " * 1500  # 1500-word text
chunks = chunk_text(sample_text, chunk_size=512, overlap=64)
assert chunks is not None, "chunk_text returned None -- did you implement it?"
assert len(chunks) > 1, f"Expected multiple chunks for a 1500-word text, got {len(chunks)}"
assert all(len(c.split()) <= 512 for c in chunks), "Some chunks exceed chunk_size"
# Check overlap: last 64 words of chunk[0] should appear at start of chunk[1]
words_0 = chunks[0].split()
words_1 = chunks[1].split()
assert words_0[-64:] == words_1[:64], "Overlap not preserved between consecutive chunks"
print(f"Chunking verified: {len(chunks)} chunks from 1500 words")
print(f"Chunk sizes: {[len(c.split()) for c in chunks]}")

In [ ]:
# --- Metadata Extraction ---
# Each chunk must retain its parent case's metadata for filtering.

def extract_metadata(case: dict, chunk_idx: int) -> dict:
    """Extract structured metadata from a case record for a given chunk.

    Args:
        case: The full case record dict
        chunk_idx: Index of the chunk within this case

    Returns:
        Dict with keys: 'case_id', 'case_name', 'court', 'jurisdiction',
        'year', 'citation', 'is_overruled', 'overruled_by', 'topic', 'chunk_idx'

    Hints:
        1. Copy the relevant metadata fields from the case dict
        2. Add chunk_idx so we can trace back to the parent case
        3. Ensure 'year' is an integer (not string)
        4. Ensure 'is_overruled' is a boolean
    """
    # TODO: Implement this function
    pass


# Verification cell
sample_meta = extract_metadata(cases[0], 0)
assert sample_meta is not None, "extract_metadata returned None"
assert "case_id" in sample_meta, "Missing 'case_id' key"
assert "jurisdiction" in sample_meta, "Missing 'jurisdiction' key"
assert isinstance(sample_meta["year"], int), "'year' should be an integer"
assert isinstance(sample_meta["is_overruled"], bool), "'is_overruled' should be a boolean"
print(f"Metadata extraction verified: {sample_meta}")

In [ ]:
# --- Build Chunk Corpus ---
# Apply chunking and metadata extraction to all cases.

all_chunks = []
all_metadata = []

for case in tqdm(cases, desc="Chunking cases"):
    chunks = chunk_text(case["text"], chunk_size=512, overlap=64)
    if chunks is None:
        continue
    for idx, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        all_metadata.append(extract_metadata(case, idx))

print(f"\nTotal chunks: {len(all_chunks)}")
print(f"Average chunks per case: {len(all_chunks)/len(cases):.1f}")
print(f"Chunks with overruled cases: {sum(1 for m in all_metadata if m['is_overruled'])}")

**Thought Questions:**
1. Why do we use overlapping chunks instead of non-overlapping splits? What information would be lost at chunk boundaries without overlap?
2. If a legal argument spans 800 words but our chunk size is 512, what happens to that argument? How could you improve the chunking strategy to preserve argumentative coherence?
3. Why is it critical that each chunk retains its parent case's metadata? What failure mode from the article does this help prevent?

---

## 3.2 Exploratory Data Analysis

Before building the retrieval pipeline, we need to understand the structure and characteristics of our corpus. This analysis will inform design decisions about chunk size, filtering strategies, and retrieval parameters.

In [ ]:
# --- Opinion Length Distribution ---

# TODO: Plot the distribution of opinion lengths (in words) across all cases.
# Use df_cases and the 'text' column.
# Include:
#   1. A histogram with 50 bins
#   2. Vertical lines for the mean and 95th percentile
#   3. Proper axis labels and title
#   4. Print the mean, median, and 95th percentile values

# Hints:
#   - Compute word counts: df_cases['text'].str.split().str.len()
#   - Use np.percentile for P95
#   - Use plt.axvline for vertical reference lines

# YOUR CODE HERE

In [ ]:
# --- Temporal Distribution ---

# TODO: Create a bar chart showing the number of opinions per court per 5-year period.
# Group years into bins: 2000-2004, 2005-2009, 2010-2014, 2015-2019, 2020-2024.
#
# Include:
#   1. Grouped or stacked bars, one color per court type
#   2. Simplify court types to: "CA Supreme", "CA Appeal", "9th Circuit", "Federal District"
#   3. Legend, axis labels, and title
#
# Hints:
#   - Create a 'period' column using pd.cut or integer division
#   - Group by period and simplified court type
#   - Use df.plot(kind='bar') or plt.bar with offsets for grouped bars

# YOUR CODE HERE

In [ ]:
# --- Citation Network Analysis ---

# TODO: Analyze the overruled case statistics:
#   1. What percentage of cases are overruled?
#   2. Is the overruled rate uniform across courts, or do some courts have higher rates?
#   3. Is there a temporal pattern (are older cases more likely to be overruled)?
#
# Create two plots:
#   (a) Bar chart of overruled rate by court type
#   (b) Line chart of overruled rate by year
#
# Hints:
#   - Group by court/year and compute mean of 'is_overruled'
#   - Use plt.subplots(1, 2, figsize=(14, 5)) for side-by-side plots

# YOUR CODE HERE

In [ ]:
# --- Topic Distribution ---

# TODO: Create a horizontal bar chart of the top 15 most common legal topics.
# Also compute: how many topics cover 80% of all cases?
#
# Hints:
#   - Use df_cases['topic'].value_counts()
#   - plt.barh for horizontal bars (easier to read long labels)
#   - Cumulative sum for the 80% threshold

# YOUR CODE HERE

**Thought Questions:**
1. Based on the opinion length distribution, what chunk size would capture 90% of legal arguments without exceeding 1,000 tokens? Show your reasoning.
2. If older cases are more likely to be overruled, how should this affect our retrieval pipeline? Should we penalize older cases in ranking, or handle this differently?
3. The topic distribution is likely imbalanced. How might this affect embedding quality for rare topics? What could you do about it?

---

## 3.3 Baseline: BM25 Keyword Search

Before building the full context engineering pipeline, we establish a baseline using BM25 — a classical keyword-matching retrieval algorithm. This gives us a performance floor to measure improvements against.

BM25 scores documents based on term frequency (TF) and inverse document frequency (IDF), without any semantic understanding. It represents the "pre-context-engineering" approach.

In [ ]:
from rank_bm25 import BM25Okapi

# Build BM25 index over all chunks
tokenized_chunks = [chunk.lower().split() for chunk in all_chunks]
bm25_index = BM25Okapi(tokenized_chunks)
print(f"BM25 index built over {len(tokenized_chunks)} chunks")

In [ ]:
# --- BM25 Retrieval and Context Assembly ---

def bm25_retrieve(query: str, bm25: BM25Okapi, chunks: List[str],
                  metadata: List[dict], top_k: int = 20) -> List[Tuple[str, dict, float]]:
    """Retrieve top-K chunks using BM25 keyword matching.

    Args:
        query: The legal query text
        bm25: A fitted BM25Okapi index
        chunks: The chunk texts (aligned with the BM25 index)
        metadata: The metadata dicts (aligned with chunks)
        top_k: Number of results to return

    Returns:
        List of (chunk_text, metadata_dict, bm25_score) tuples, sorted by score descending

    Hints:
        1. Tokenize the query the same way as the corpus: query.lower().split()
        2. Use bm25.get_scores(tokenized_query) to get all scores
        3. Use np.argsort to find top-K indices
        4. Return tuples of (chunk, metadata, score)
    """
    # TODO: Implement this function
    pass


def assemble_naive_context(system_prompt: str, query: str,
                           retrieved: List[Tuple[str, dict, float]],
                           max_tokens: int = 93000) -> str:
    """Assemble a naive context by concatenating all retrieved chunks.

    This represents the baseline approach: no filtering, no reranking,
    no strategic placement. Just dump everything in.

    Args:
        system_prompt: The system instruction text
        query: The user's legal query
        retrieved: List of (chunk, metadata, score) from bm25_retrieve
        max_tokens: Maximum context tokens

    Returns:
        Assembled context string

    Hints:
        1. Start with the system prompt
        2. Concatenate all retrieved chunks (in retrieval order)
        3. Append the query at the end
        4. Use len(text) // 4 as a rough token estimate
        5. Truncate if over budget (just cut off excess chunks)
    """
    # TODO: Implement this function
    pass


# Verification
sample_query = "Does California law recognize strict liability for defective products?"
sample_results = bm25_retrieve(sample_query, bm25_index, all_chunks, all_metadata, top_k=20)
assert sample_results is not None, "bm25_retrieve returned None"
assert len(sample_results) <= 20, f"Expected at most 20 results, got {len(sample_results)}"
print(f"BM25 retrieved {len(sample_results)} chunks for query: '{sample_query}'")
print(f"Top result score: {sample_results[0][2]:.4f}")
print(f"Top result jurisdiction: {sample_results[0][1]['jurisdiction']}")
print(f"Top result court: {sample_results[0][1]['court']}")

In [ ]:
# --- Evaluate Baseline ---

# Test queries representing different legal research tasks
TEST_QUERIES = [
    {"query": "Does California recognize strict liability for defective products?",
     "target_jurisdiction": "california", "target_topic": "product liability"},
    {"query": "What is the standard for wrongful termination in violation of public policy?",
     "target_jurisdiction": "california", "target_topic": "wrongful termination"},
    {"query": "When can a court compel arbitration of employment disputes?",
     "target_jurisdiction": "california", "target_topic": "arbitration"},
    {"query": "What constitutes intentional infliction of emotional distress under California law?",
     "target_jurisdiction": "california", "target_topic": "emotional distress"},
    {"query": "What is the statute of limitations for fraud claims in California?",
     "target_jurisdiction": "california", "target_topic": "statute of limitations"},
    {"query": "Under what circumstances can a landlord evict a tenant in California?",
     "target_jurisdiction": "california", "target_topic": "landlord-tenant"},
    {"query": "What is the standard for granting summary judgment in federal court?",
     "target_jurisdiction": "federal-9th", "target_topic": "civil rights"},
    {"query": "How does the Ninth Circuit evaluate Fourth Amendment search claims?",
     "target_jurisdiction": "federal-9th", "target_topic": "fourth amendment"},
    {"query": "What damages are available for employment discrimination under FEHA?",
     "target_jurisdiction": "california", "target_topic": "employment discrimination"},
    {"query": "What is the test for unfair business practices under California B&P Code 17200?",
     "target_jurisdiction": "california", "target_topic": "unfair business practices"},
]


def evaluate_baseline(queries: List[dict], bm25: BM25Okapi,
                      chunks: List[str], metadata: List[dict],
                      top_k: int = 10) -> dict:
    """Evaluate BM25 baseline on test queries.

    Compute:
    - Jurisdictional precision: % of retrieved chunks from target jurisdiction
    - Topic relevance: % of retrieved chunks matching target topic
    - Overruled rate: % of retrieved chunks from overruled cases
    - Context utilization: % of retrieved tokens that are "relevant"
      (matching jurisdiction AND topic)

    Args:
        queries: List of test query dicts with 'query', 'target_jurisdiction', 'target_topic'
        bm25: Fitted BM25 index
        chunks: Chunk texts
        metadata: Chunk metadata
        top_k: Number of chunks to retrieve per query

    Returns:
        Dict with average metrics across all queries

    Hints:
        1. For each query, call bm25_retrieve
        2. Check each result's metadata against target_jurisdiction and target_topic
        3. Compute the four metrics per query, then average across queries
        4. Also track per-query results for later analysis
    """
    # TODO: Implement this function
    pass


# Run evaluation
baseline_metrics = evaluate_baseline(TEST_QUERIES, bm25_index, all_chunks, all_metadata)
assert baseline_metrics is not None, "evaluate_baseline returned None"
print("\n=== BM25 Baseline Metrics ===")
for metric, value in baseline_metrics.items():
    print(f"  {metric}: {value:.3f}")

**Thought Questions:**
1. Why does BM25 struggle with legal text? Identify two specific failure examples from your results (e.g., a query where the wrong jurisdiction dominated, or a topically irrelevant result scored highly).
2. What is the overruled rate in your baseline retrievals? This directly corresponds to the "context poisoning" failure mode. How significant is this problem?
3. What is the context utilization ratio? If only 40% of retrieved tokens are relevant, what does that imply about the "context confusion" failure mode?

---

## 3.4 Context Engineering Pipeline

Now we build the full pipeline. Each stage addresses one or more of the four context failure modes using the corresponding strategy from the article.

### Stage 1: Query Analysis (Select Strategy)

In [ ]:
@dataclass
class LegalQuery:
    """Structured representation of a legal research query."""
    legal_question: str
    jurisdiction_filter: List[str]
    date_range: Tuple[int, int]
    strategy_keywords: List[str]
    excluded_topics: List[str]

# Jurisdiction mapping: code -> list of acceptable court patterns
JURISDICTION_MAP = {
    "california": ["Supreme Court of California", "California Court of Appeal"],
    "federal-9th": ["Ninth Circuit", "Northern District of California",
                    "Central District of California"],
    "CA": ["Supreme Court of California", "California Court of Appeal"],
    "9th-circuit": ["Ninth Circuit"],
}

# Combined: California queries should also include relevant federal
COMBINED_JURISDICTIONS = {
    "california": ["Supreme Court of California", "California Court of Appeal",
                   "Ninth Circuit", "Northern District of California",
                   "Central District of California"],
}


def parse_legal_query(
    query_text: str,
    jurisdiction: str,
    strategy_notes: str = "",
    date_range: Optional[Tuple[int, int]] = None
) -> LegalQuery:
    """Parse raw attorney input into a structured legal query.

    Args:
        query_text: The legal question or research request
        jurisdiction: Target jurisdiction (e.g., "california", "federal-9th")
        strategy_notes: Attorney's strategy preferences, comma-separated keywords
        date_range: Tuple of (start_year, end_year) or None for default (2000, 2024)

    Returns:
        LegalQuery with parsed and validated fields

    Hints:
        1. Validate jurisdiction against JURISDICTION_MAP keys
        2. If jurisdiction is 'california', use COMBINED_JURISDICTIONS to include
           both state and relevant federal courts
        3. Parse strategy_notes by splitting on commas and stripping whitespace
        4. Default date_range to (2000, 2024) if not provided
        5. excluded_topics can be extracted from strategy_notes if prefixed with "not:"
           (e.g., "strict liability, not: criminal")
    """
    # TODO: Implement this function
    pass


# Verification
test_query = parse_legal_query(
    "Does California recognize strict liability for defective products?",
    "california",
    "strict liability, product defects, not: criminal",
    (2010, 2024)
)
assert test_query is not None, "parse_legal_query returned None"
assert len(test_query.jurisdiction_filter) > 0, "No jurisdiction filters set"
assert test_query.date_range == (2010, 2024), f"Date range incorrect: {test_query.date_range}"
assert "strict liability" in test_query.strategy_keywords, "Strategy keywords not parsed"
print(f"Query parsed successfully:")
print(f"  Question: {test_query.legal_question}")
print(f"  Jurisdictions: {test_query.jurisdiction_filter}")
print(f"  Date range: {test_query.date_range}")
print(f"  Strategy keywords: {test_query.strategy_keywords}")
print(f"  Excluded topics: {test_query.excluded_topics}")

### Stage 2: Filtered Retrieval (Select Strategy — Hard Filters)

In [ ]:
def filter_corpus(
    chunks: List[str],
    metadata: List[dict],
    query: LegalQuery
) -> Tuple[List[str], List[dict]]:
    """Filter corpus chunks by jurisdiction, date, and validity.

    This is the first line of defense against context confusion and
    context poisoning. By filtering BEFORE retrieval, we ensure that
    irrelevant jurisdictions and overruled cases never enter the
    candidate pool.

    Args:
        chunks: List of text chunks
        metadata: List of metadata dicts (aligned with chunks)
        query: Parsed LegalQuery with filter criteria

    Returns:
        Tuple of (filtered_chunks, filtered_metadata)

    Hints:
        1. For each chunk, check if its court name contains any of the
           jurisdiction_filter patterns (use substring matching)
        2. Check if the case year falls within query.date_range
        3. CRITICAL: Exclude any chunk where is_overruled is True
        4. Keep both the chunk and its metadata in sync
        5. Print the reduction ratio for debugging
    """
    # TODO: Implement this function
    pass


# Verification
test_lq = parse_legal_query(
    "strict liability for defective products",
    "california",
    date_range=(2010, 2024)
)
filtered_chunks, filtered_meta = filter_corpus(all_chunks, all_metadata, test_lq)
assert filtered_chunks is not None, "filter_corpus returned None"
assert len(filtered_chunks) < len(all_chunks), "Filtering should reduce corpus size"
assert all(not m["is_overruled"] for m in filtered_meta), "Overruled cases should be excluded"
assert all(m["year"] >= 2010 for m in filtered_meta), "Cases before 2010 should be excluded"
print(f"Filtering results:")
print(f"  Original: {len(all_chunks)} chunks")
print(f"  Filtered: {len(filtered_chunks)} chunks")
print(f"  Reduction: {(1 - len(filtered_chunks)/len(all_chunks))*100:.1f}%")
print(f"  Jurisdictions present: {set(m['jurisdiction'] for m in filtered_meta)}")

### Stage 3: Semantic Search + Reranking (Select Strategy — Soft Ranking)

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder

# Load models (these are small enough for Colab)
print("Loading embedding model...")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # 22M params, fast
print("Loading reranker model...")
reranker_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')  # 22M params
print("Models loaded.")

In [ ]:
def embed_and_retrieve(
    query_text: str,
    chunks: List[str],
    model: SentenceTransformer,
    top_k: int = 20
) -> List[Tuple[str, float, int]]:
    """Retrieve top-K chunks by cosine similarity using sentence embeddings.

    Args:
        query_text: The legal query text
        chunks: Filtered corpus chunks
        model: A SentenceTransformer model for encoding
        top_k: Number of candidates to retrieve

    Returns:
        List of (chunk_text, similarity_score, original_index) tuples,
        sorted by score descending

    Hints:
        1. Encode the query: model.encode(query_text, normalize_embeddings=True)
        2. Encode all chunks: model.encode(chunks, normalize_embeddings=True,
           show_progress_bar=True, batch_size=64)
        3. Compute cosine similarities via dot product (already normalized)
        4. Use np.argsort(scores)[::-1][:top_k] for top-K indices
        5. Return chunk text, score, and original index (for metadata lookup)
    """
    # TODO: Implement this function
    pass


def rerank(
    query_text: str,
    candidates: List[Tuple[str, float, int]],
    reranker: CrossEncoder,
    top_n: int = 10
) -> List[Tuple[str, float, int]]:
    """Rerank candidates using a cross-encoder for fine-grained relevance.

    The cross-encoder sees both query and document together, allowing it
    to capture nuanced relevance signals that the bi-encoder misses.

    Args:
        query_text: The legal query text
        candidates: List of (chunk_text, initial_score, original_index)
        reranker: A CrossEncoder model
        top_n: Number of final results to return

    Returns:
        List of (chunk_text, reranker_score, original_index) tuples,
        sorted by reranker score descending

    Hints:
        1. Create input pairs: [(query_text, chunk_text) for chunk_text, _, _ in candidates]
        2. Get scores: reranker.predict(pairs)
        3. Sort by reranker scores (descending)
        4. Return top_n results with reranker scores replacing initial scores
    """
    # TODO: Implement this function
    pass


# Verification
if filtered_chunks:
    embed_results = embed_and_retrieve(
        "strict liability for defective products",
        filtered_chunks, embed_model, top_k=20
    )
    assert embed_results is not None, "embed_and_retrieve returned None"
    assert len(embed_results) == min(20, len(filtered_chunks))
    print(f"Embedding retrieval: {len(embed_results)} candidates")
    print(f"Score range: {embed_results[-1][1]:.4f} to {embed_results[0][1]:.4f}")

    reranked = rerank(
        "strict liability for defective products",
        embed_results, reranker_model, top_n=10
    )
    assert reranked is not None, "rerank returned None"
    assert len(reranked) == min(10, len(embed_results))
    print(f"\nAfter reranking: {len(reranked)} results")
    print(f"Reranker score range: {reranked[-1][1]:.4f} to {reranked[0][1]:.4f}")

### Stage 4: Context Assembly (All Four Strategies)

In [ ]:
class ContextEngine:
    """Assembles optimal context for legal brief generation.

    Implements all four context engineering strategies:
    - Write: Loads persisted case validity index and attorney preferences
    - Select: Uses filtered retrieval and reranking results
    - Compress: Truncates history to fit token budget, keeping most recent
    - Isolate: Structures context with clear XML boundaries
    """

    def __init__(self, max_tokens: int = 128000, reserved_output: int = 35000):
        self.max_tokens = max_tokens
        self.reserved = reserved_output
        self.available = max_tokens - reserved_output

    def estimate_tokens(self, text: str) -> int:
        """Estimate token count for a text string.

        Hints:
            1. Use len(text) // 4 as a rough heuristic
            2. Legal text is slightly denser: consider len(text) // 3.8
            3. Round up to be conservative
        """
        # TODO: Implement this method
        pass

    def allocate_budget(
        self,
        system_tokens: int,
        facts_tokens: int,
        strategy_tokens: int,
        num_rag_docs: int,
        avg_doc_tokens: int
    ) -> dict:
        """Dynamically allocate token budget across context components.

        Args:
            system_tokens: Tokens for system prompt (fixed)
            facts_tokens: Tokens for case facts (fixed)
            strategy_tokens: Tokens for strategy notes (fixed)
            num_rag_docs: Number of retrieved documents available
            avg_doc_tokens: Average tokens per retrieved document

        Returns:
            Dict with keys: 'system', 'facts', 'strategy', 'rag',
            'history', 'buffer' and their token allocations

        Hints:
            1. Fixed allocations: system + facts + strategy
            2. Remaining = self.available - fixed
            3. RAG budget = min(num_rag_docs * avg_doc_tokens, remaining * 0.6)
            4. History budget = min(remaining * 0.25, remaining - rag_budget)
            5. Buffer = remaining - rag - history (at least 5% of remaining)
            6. If RAG under-utilizes its budget, give excess to history
        """
        # TODO: Implement this method
        pass

    def compress_history(self, history: List[str], budget_tokens: int) -> List[str]:
        """Keep most recent history within token budget.

        Args:
            history: List of conversation messages (oldest first)
            budget_tokens: Maximum tokens for history

        Returns:
            List of retained messages (chronological order)

        Hints:
            1. Iterate from most recent (reversed) and accumulate tokens
            2. Stop when adding the next message would exceed budget
            3. Reverse back to chronological order before returning
        """
        # TODO: Implement this method
        pass

    def assemble(
        self,
        system_prompt: str,
        case_facts: str,
        strategy_notes: str,
        retrieved_docs: List[Tuple[str, float]],
        history: List[str],
        memory: List[str]
    ) -> str:
        """Assemble the complete context with XML-tagged structure.

        Args:
            system_prompt: System instructions for the LLM
            case_facts: The case facts provided by the attorney
            strategy_notes: The attorney's strategy preferences
            retrieved_docs: List of (chunk_text, relevance_score) tuples
            history: Conversation history messages
            memory: Persisted memory items (from prior sessions)

        Returns:
            Complete context string within token budget

        Hints:
            1. Compute token estimates for fixed components
            2. Call allocate_budget to get per-component limits
            3. Compress history to fit its budget
            4. Truncate retrieved_docs to fit RAG budget (keep highest-scored)
            5. IMPORTANT: Place the #1 and #2 most relevant docs at the
               BEGINNING. Place the #3 most relevant doc at the END.
               Fill middle positions with remaining docs.
               This mitigates the "Lost in the Middle" effect.
            6. Format with XML tags: <system>, <memory>, <case_facts>,
               <strategy>, <history>, <retrieved_authority>, <query>
            7. Include relevance scores in the retrieved section
            8. Verify total tokens <= self.available
        """
        # TODO: Implement this method
        pass


# Verification
engine = ContextEngine(max_tokens=128000, reserved_output=35000)

# Test token estimation
test_tokens = engine.estimate_tokens("This is a test sentence with about ten words.")
assert test_tokens is not None, "estimate_tokens returned None"
assert 5 < test_tokens < 20, f"Token estimate seems off: {test_tokens}"
print(f"Token estimation: '{test_tokens}' tokens for a ~10-word sentence")

# Test budget allocation
budget = engine.allocate_budget(
    system_tokens=500, facts_tokens=1000, strategy_tokens=200,
    num_rag_docs=10, avg_doc_tokens=512
)
assert budget is not None, "allocate_budget returned None"
assert sum(budget.values()) <= engine.available, "Budget exceeds available tokens"
print(f"\nBudget allocation: {budget}")
print(f"Total allocated: {sum(budget.values())} / {engine.available} available")

# Test full assembly
sample_context = engine.assemble(
    system_prompt="You are a legal research assistant for California litigation.",
    case_facts="The plaintiff alleges product defect in a consumer device.",
    strategy_notes="Argue strict liability under California law.",
    retrieved_docs=[("Sample legal text " * 50, 0.92), ("Another case " * 50, 0.87)],
    history=["Previous turn about case background"],
    memory=["Attorney prefers concise briefs", "California jurisdiction focus"]
)
assert sample_context is not None, "assemble returned None"
assert "<system>" in sample_context, "Missing <system> tag"
assert "<retrieved_authority>" in sample_context, "Missing <retrieved_authority> tag"
total_tokens = engine.estimate_tokens(sample_context)
assert total_tokens <= engine.available, f"Context exceeds budget: {total_tokens} > {engine.available}"
print(f"\nAssembled context: {total_tokens} tokens")
print(f"First 500 chars:\n{sample_context[:500]}")

### Stage 5: Citation Verification (Write Strategy)

In [ ]:
def build_validity_index(metadata: List[dict]) -> dict:
    """Build a citation validity lookup from corpus metadata.

    This implements the 'Write' strategy: we persist a lookup table
    of citation validity OUTSIDE the context window. Every generated
    citation is checked against this index before being included.

    Args:
        metadata: List of chunk metadata dicts

    Returns:
        Dict mapping citation string -> {
            'status': 'valid' | 'overruled' | 'superseded',
            'overruled_by': str or None,
            'year': int,
            'case_name': str
        }

    Hints:
        1. Iterate through metadata, keyed by 'citation'
        2. For overruled cases, set status='overruled' and include 'overruled_by'
        3. For valid cases, set status='valid'
        4. Handle duplicates: if the same citation appears multiple times
           (from multiple chunks), keep the entry with the most information
        5. This should be a simple dictionary lookup -- O(1) per citation check
    """
    # TODO: Implement this function
    pass


def verify_citations(
    generated_text: str,
    validity_index: dict
) -> List[dict]:
    """Extract and verify all case citations in generated text.

    Args:
        generated_text: The LLM-generated legal brief text
        validity_index: The citation validity lookup from build_validity_index

    Returns:
        List of dicts, each with:
        - 'citation': the extracted citation string
        - 'status': 'valid', 'overruled', or 'unverified'
        - 'found_in_index': bool
        - 'recommendation': 'keep', 'flag', or 'remove'
        - 'details': additional info (e.g., overruling case)

    Hints:
        1. Use regex to extract citations matching legal citation patterns:
           r'\\d+\\s+(?:Cal\\.(?:3d|4th|5th)|Cal\\.App\\.(?:4th|5th)|F\\.3d|F\\.Supp\\.(?:2d|3d))\\s+\\d+\\s+\\(\\d{4}\\)'
        2. For each extracted citation, look it up in the validity index
        3. If found and valid -> recommendation='keep'
        4. If found and overruled -> recommendation='remove', include overruling case
        5. If not found -> recommendation='flag' (unverified)
    """
    # TODO: Implement this function
    pass


# Verification
validity_idx = build_validity_index(all_metadata)
assert validity_idx is not None, "build_validity_index returned None"
assert len(validity_idx) > 0, "Validity index is empty"

# Count overruled entries
overruled_count = sum(1 for v in validity_idx.values() if v['status'] == 'overruled')
print(f"Validity index built: {len(validity_idx)} unique citations")
print(f"  Valid: {sum(1 for v in validity_idx.values() if v['status'] == 'valid')}")
print(f"  Overruled: {overruled_count}")

# Test citation verification on synthetic text
sample_brief = "The court in " + list(validity_idx.keys())[0] + " established that..."
results = verify_citations(sample_brief, validity_idx)
assert results is not None, "verify_citations returned None"
print(f"\nCitation verification test: {len(results)} citations found")
for r in results:
    print(f"  {r['citation']} -> {r['recommendation']}")

**Thought Questions:**
1. The citation verification step happens AFTER generation. Could we prevent bad citations from being generated in the first place? How would you modify the pipeline?
2. What if a citation is valid but from the wrong jurisdiction? The validity index only tracks overruled status. How would you extend it to also flag jurisdictional mismatches?
3. The "Write" strategy persists the validity index outside the context window. What other information should be persisted? Think about what an attorney would want remembered across sessions.

---

## 3.5 Training: Embedding Model Fine-tuning

In this section, we fine-tune the embedding model to improve retrieval quality on legal text. The key insight: a general-purpose embedding model does not understand that "battery" (the tort) and "battery" (the device) should be far apart in embedding space when the query is about personal injury law.

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# --- Generate Training Pairs ---
# We create query-passage pairs from our corpus.
# Positive pairs: query about a topic matched with a chunk on that topic
# Hard negatives: chunks that are semantically similar but wrong topic/jurisdiction

def generate_training_pairs(
    chunks: List[str],
    metadata: List[dict],
    num_pairs: int = 500
) -> List[dict]:
    """Generate training pairs for embedding fine-tuning.

    Each pair consists of:
    - query: a synthetic legal query about a topic
    - positive: a chunk that matches the query's topic and jurisdiction
    - hard_negatives: 3 chunks that are topically similar but wrong
      jurisdiction or wrong subtopic

    Args:
        chunks: All corpus chunks
        metadata: Aligned metadata
        num_pairs: Number of training pairs to generate

    Returns:
        List of dicts with keys: 'query', 'positive', 'hard_negatives'

    Hints:
        1. Group chunks by topic using metadata
        2. For each pair, randomly select a topic and jurisdiction
        3. Create a query template: f"What does {jurisdiction} law say about {topic}?"
        4. Positive: random chunk matching both topic and jurisdiction
        5. Hard negatives: chunks matching topic but WRONG jurisdiction (hardest),
           or matching jurisdiction but WRONG topic (easier negative)
        6. Include 2 wrong-jurisdiction and 1 wrong-topic negatives per pair
    """
    # TODO: Implement this function
    pass


training_pairs = generate_training_pairs(all_chunks, all_metadata, num_pairs=500)
assert training_pairs is not None, "generate_training_pairs returned None"
assert len(training_pairs) > 0, "No training pairs generated"
print(f"Generated {len(training_pairs)} training pairs")
print(f"Sample pair:")
print(f"  Query: {training_pairs[0]['query'][:100]}...")
print(f"  Positive: {training_pairs[0]['positive'][:100]}...")
print(f"  Num hard negatives: {len(training_pairs[0]['hard_negatives'])}")

In [ ]:
# --- InfoNCE Contrastive Loss ---

def info_nce_loss(query_emb, positive_emb, negative_embs, temperature=0.07):
    """Compute InfoNCE contrastive loss.

    This loss pulls the query embedding closer to the positive document
    and pushes it away from negative documents in embedding space.

    Args:
        query_emb: Query embedding tensor, shape (batch_size, embed_dim)
        positive_emb: Positive document embedding, shape (batch_size, embed_dim)
        negative_embs: Negative document embeddings, shape (batch_size, num_neg, embed_dim)
        temperature: Temperature parameter (controls sharpness of distribution)

    Returns:
        Scalar loss value

    Hints:
        1. Compute positive similarity: (query_emb * positive_emb).sum(dim=-1) / temperature
        2. Compute negative similarities: torch.bmm(negative_embs, query_emb.unsqueeze(-1)).squeeze(-1) / temperature
        3. Concatenate positive and negative similarities: logits shape (batch_size, 1 + num_neg)
        4. Labels are all zeros (positive is at index 0)
        5. Use F.cross_entropy(logits, labels)

    Mathematical reference:
        L = -log(exp(sim(q, d+) / tau) / (exp(sim(q, d+) / tau) + sum(exp(sim(q, d-) / tau))))
    """
    # TODO: Implement this function
    pass


# Verification with random tensors
batch_q = torch.randn(4, 384)  # batch of 4 queries, 384-dim embeddings
batch_p = torch.randn(4, 384)  # 4 positive docs
batch_n = torch.randn(4, 3, 384)  # 3 negatives per query
# Normalize
batch_q = F.normalize(batch_q, dim=-1)
batch_p = F.normalize(batch_p, dim=-1)
batch_n = F.normalize(batch_n, dim=-1)

loss = info_nce_loss(batch_q, batch_p, batch_n)
assert loss is not None, "info_nce_loss returned None"
assert loss.item() > 0, "Loss should be positive"
print(f"InfoNCE loss on random embeddings: {loss.item():.4f}")
print(f"Expected: ~log(4) = {np.log(4):.4f} for random vectors")

In [ ]:
# --- Fine-tuning Loop ---
# Fine-tune the embedding model on legal domain data.

# TODO: Implement the fine-tuning loop.
#
# Structure:
# 1. Create a simple Dataset class that yields (query, positive, negatives) tuples
# 2. Use a DataLoader with batch_size=8
# 3. For each batch:
#    a. Encode queries, positives, and negatives using embed_model.encode()
#    b. Convert to tensors
#    c. Compute InfoNCE loss
#    d. Backpropagate and update (use AdamW optimizer, lr=2e-5)
# 4. Train for 3 epochs
# 5. Plot the training loss curve
#
# Note: For full fine-tuning you would use the sentence-transformers
# training API. Here we demonstrate the loss computation and training
# dynamics conceptually.
#
# Hints:
#   - For Colab efficiency, you can fine-tune only the last 2 layers
#   - Log loss every 10 steps
#   - Save the loss values for plotting

# YOUR CODE HERE

In [ ]:
# --- Compare Before/After Fine-tuning ---

# TODO: Run the embedding retrieval (embed_and_retrieve) on the test queries
# using both the original and fine-tuned models. Compare Relevance@10 (using
# the topic and jurisdiction matching from the baseline evaluation).
#
# Create a bar chart comparing:
#   - BM25 baseline topic relevance
#   - Original embedding model topic relevance
#   - Fine-tuned embedding model topic relevance (if fine-tuning was run)
#
# Hints:
#   - Reuse the evaluate_baseline pattern but with embed_and_retrieve instead of bm25_retrieve
#   - You may need to re-filter the corpus for each query first

# YOUR CODE HERE

**Thought Questions:**
1. Why do we use hard negatives (wrong jurisdiction, similar topic) rather than random negatives? How does this affect what the embedding model learns?
2. The temperature parameter $\tau$ in InfoNCE is set to 0.07. What happens if you increase it to 0.5? What if you decrease it to 0.01? Think about the gradient magnitude.
3. We only fine-tune on 500 pairs. In production, BriefEngine would use thousands of pairs from attorney feedback. How would you collect high-quality training data from real attorney usage without burdening the attorneys?

---

## 3.6 Evaluation

Now we evaluate the complete pipeline (filtering + embedding retrieval + reranking + context assembly) against the BM25 baseline.

In [ ]:
def evaluate_full_pipeline(
    queries: List[dict],
    all_chunks: List[str],
    all_metadata: List[dict],
    embed_model: SentenceTransformer,
    reranker: CrossEncoder,
    top_k_retrieval: int = 20,
    top_n_rerank: int = 10
) -> dict:
    """Evaluate the full context engineering pipeline on test queries.

    For each query:
    1. Parse the query into a LegalQuery
    2. Filter the corpus by jurisdiction and date
    3. Embed and retrieve top-K candidates
    4. Rerank to top-N final results
    5. Compute metrics: jurisdictional precision, topic relevance,
       overruled rate, context utilization

    Args:
        queries: List of test query dicts
        all_chunks, all_metadata: Full corpus
        embed_model: SentenceTransformer for retrieval
        reranker: CrossEncoder for reranking
        top_k_retrieval: Candidates from embedding search
        top_n_rerank: Final results after reranking

    Returns:
        Dict with average metrics and per-query results

    Hints:
        1. For each query, call parse_legal_query first
        2. Then filter_corpus to narrow candidates
        3. Then embed_and_retrieve on filtered set
        4. Then rerank the candidates
        5. Compute the same metrics as evaluate_baseline for fair comparison
        6. Also measure: latency per query (time each stage)
    """
    # TODO: Implement this function
    pass


# Run evaluation
pipeline_metrics = evaluate_full_pipeline(
    TEST_QUERIES, all_chunks, all_metadata, embed_model, reranker_model
)
assert pipeline_metrics is not None, "evaluate_full_pipeline returned None"
print("\n=== Full Pipeline Metrics ===")
for metric, value in pipeline_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.3f}")

In [ ]:
# --- Comparison Table ---

# TODO: Create a comparison table (as a pandas DataFrame) showing
# baseline vs. pipeline metrics side by side.
#
# Columns: Metric | BM25 Baseline | Full Pipeline | Improvement
# Rows: Jurisdictional Precision, Topic Relevance, Overruled Rate,
#        Context Utilization
#
# Also create a grouped bar chart visualizing the comparison.
#
# Hints:
#   - Use pd.DataFrame with the metrics dicts
#   - Improvement = (pipeline - baseline) / baseline * 100 for increases
#   - For overruled rate, improvement = (baseline - pipeline) / baseline * 100 (lower is better)

# YOUR CODE HERE

In [ ]:
# --- Precision-Recall Curve ---

# TODO: Plot a precision-recall curve for the retrieval component.
#
# For each query:
# 1. Retrieve top-50 candidates via embedding search
# 2. At each cutoff K (1, 2, 3, ..., 50), compute:
#    - Precision@K = (relevant docs in top K) / K
#    - Recall@K = (relevant docs in top K) / (total relevant docs)
# 3. Plot the average P-R curve across all test queries
#
# Also mark the operating point (K=10 after reranking) on the curve.
#
# Hints:
#   - "Relevant" means matching the target_topic from the test query
#   - Use matplotlib to plot with labeled axes
#   - The curve should show the precision-recall tradeoff clearly

# YOUR CODE HERE

**Thought Questions:**
1. By how much did the pipeline improve over the BM25 baseline on each metric? Which improvement was largest? Why?
2. What is the overruled rate in the full pipeline results? If it is not 0%, explain what went wrong — the filtering should have excluded all overruled cases.
3. Look at the precision-recall curve. Is there a clear "elbow" point? What does this suggest about the optimal number of documents to include in context?

---

## 3.7 Error Analysis

Systematic error analysis is critical for improving the pipeline. We categorize errors into the four failure modes from the article.

In [ ]:
def analyze_errors(
    queries: List[dict],
    pipeline_results: dict,
    all_metadata: List[dict]
) -> dict:
    """Categorize pipeline errors into the four context failure modes.

    For each query result, check for:
    1. Poisoning: Any overruled case made it into final results
    2. Distraction: A highly relevant document was in the filtered corpus
       but did not appear in the top-N results (false negative)
    3. Confusion: An irrelevant document (wrong topic) appeared in top-N
    4. Clash: Retrieved documents contain conflicting legal conclusions
       on the same issue

    Args:
        queries: Test query dicts
        pipeline_results: Output from evaluate_full_pipeline (includes per-query results)
        all_metadata: Full corpus metadata (for checking what was missed)

    Returns:
        Dict with counts and examples for each failure mode:
        {
            'poisoning': {'count': int, 'examples': [...]},
            'distraction': {'count': int, 'examples': [...]},
            'confusion': {'count': int, 'examples': [...]},
            'clash': {'count': int, 'examples': [...]},
        }

    Hints:
        1. For poisoning: check is_overruled in result metadata
        2. For distraction: check if any chunk with matching topic was in
           the filtered set but NOT in the final top-N
        3. For confusion: check if any top-N result has a non-matching topic
        4. For clash: this is harder to detect automatically. Use a heuristic:
           if two retrieved chunks are about the same topic but from different
           court levels (e.g., trial vs. appellate), flag as potential clash
    """
    # TODO: Implement this function
    pass


# Run error analysis
# Note: This requires per-query results from evaluate_full_pipeline.
# If your evaluate_full_pipeline stores per-query data, use it here.
# Otherwise, re-run the pipeline with per-query result tracking.

# YOUR CODE HERE

# TODO: Print a summary of error counts by failure mode
# TODO: For the most frequent failure mode, show 2 detailed examples
# TODO: Answer: Which failure mode is hardest to detect automatically? Why?

**Thought Questions:**
1. Which failure mode was most common in your pipeline? Was this expected based on the pipeline design?
2. Which failure mode is hardest to detect automatically? Propose a method for detecting it that goes beyond simple heuristics.
3. For each of the top 3 failure modes, propose one specific pipeline change that would reduce its frequency. Be concrete — name the function you would modify and what you would change.

---

## 3.8 Scalability and Deployment

Profile the pipeline to understand latency characteristics and identify optimization opportunities.

In [ ]:
def profile_pipeline(
    query: str,
    jurisdiction: str,
    all_chunks: List[str],
    all_metadata: List[dict],
    embed_model: SentenceTransformer,
    reranker: CrossEncoder,
    num_runs: int = 5
) -> dict:
    """Profile each stage of the pipeline for latency.

    Run the full pipeline num_runs times and report:
    - Mean and P95 latency for each stage
    - Total end-to-end latency

    Stages to profile:
    1. Query parsing
    2. Corpus filtering
    3. Embedding + retrieval
    4. Reranking
    5. Context assembly

    Args:
        query: Test query string
        jurisdiction: Target jurisdiction
        all_chunks, all_metadata: Full corpus
        embed_model: SentenceTransformer
        reranker: CrossEncoder
        num_runs: Number of profiling runs

    Returns:
        Dict mapping stage_name -> {'mean_ms': float, 'p95_ms': float}

    Hints:
        1. Use time.perf_counter() for high-resolution timing
        2. Run each stage independently within the loop
        3. Store per-run timings in a list, then compute mean and P95
        4. P95 = np.percentile(timings, 95)
    """
    # TODO: Implement this function
    pass


# Run profiling
profile_results = profile_pipeline(
    "Does California recognize strict liability for defective products?",
    "california",
    all_chunks, all_metadata, embed_model, reranker_model
)
assert profile_results is not None, "profile_pipeline returned None"
print("\n=== Pipeline Latency Profile ===")
print(f"{'Stage':<25} {'Mean (ms)':<15} {'P95 (ms)':<15}")
print("-" * 55)
total_mean = 0
for stage, timings in profile_results.items():
    print(f"{stage:<25} {timings['mean_ms']:<15.1f} {timings['p95_ms']:<15.1f}")
    total_mean += timings['mean_ms']
print("-" * 55)
print(f"{'TOTAL':<25} {total_mean:<15.1f}")

In [ ]:
# --- Optimization Analysis ---

# TODO: Based on the profiling results:
# 1. Identify the top 2 latency bottlenecks
# 2. For each bottleneck, propose a specific optimization and estimate
#    the expected speedup
# 3. Implement ONE optimization (the simpler one) and re-profile to
#    measure the actual improvement
#
# Common optimizations to consider:
#   - Pre-compute and cache embeddings for the full corpus
#   - Batch encoding instead of single-document encoding
#   - Use approximate nearest neighbor (FAISS) instead of brute-force search
#   - Reduce the number of reranking candidates
#
# Hints:
#   - If embedding is the bottleneck, try pre-computing corpus embeddings once
#   - If reranking is the bottleneck, try reducing top_k from 20 to 10
#   - Measure the impact on both latency AND retrieval quality

# YOUR CODE HERE

**Thought Questions:**
1. Which stage dominates latency? Is this consistent with BriefEngine's latency budget from the case study?
2. Pre-computing corpus embeddings trades storage for latency. How much storage would 42 million 1024-dimensional float32 embeddings require? Is this feasible?
3. If you reduce the reranking candidates from 20 to 10, how does retrieval quality change? Is the latency reduction worth the quality tradeoff?

---

## 3.9 Ethical and Regulatory Analysis

Legal AI raises unique ethical challenges that require careful consideration.

In [ ]:
# --- Temporal Bias Analysis ---

# TODO: Investigate whether the retrieval pipeline has a temporal bias.
# Run all 10 test queries through the full pipeline and collect the
# publication years of all retrieved documents.
#
# Create:
# 1. A histogram of retrieved document years (across all queries)
# 2. Compare it against the corpus distribution of years
# 3. Compute the KL divergence between retrieved and corpus distributions
#
# If there is a bias toward older or newer cases, explain:
# - Why it might exist (training data distribution? embedding space properties?)
# - Whether it is desirable or harmful for legal research
# - How you would correct it if harmful
#
# Hints:
#   - Collect years from the metadata of all retrieved docs
#   - Normalize both distributions to get probability distributions
#   - KL(P||Q) = sum(P(x) * log(P(x) / Q(x))) for each year bin
#   - Use 5-year bins for smoother distributions

# YOUR CODE HERE

In [ ]:
# --- Ethical Impact Assessment ---

# TODO: Write a 500-word ethical impact assessment for BriefEngine's
# AI legal research assistant. Address these four questions:
#
# (a) Who benefits from this system?
# (b) Who could be harmed?
# (c) What safeguards should be in place?
# (d) Should this system be deployed without mandatory attorney review?
#
# Write your assessment as a multi-line string below.
# This is a writing exercise, not a coding exercise.

ethical_assessment = """
# Ethical Impact Assessment: BriefEngine AI Legal Research Assistant

## (a) Who Benefits

[YOUR ASSESSMENT HERE - discuss attorneys at mid-size firms, their clients,
access to justice implications, efficiency gains]

## (b) Who Could Be Harmed

[YOUR ASSESSMENT HERE - discuss opposing parties if briefs contain errors,
clients who over-rely on AI, junior attorneys whose skills may atrophy,
communities affected by biased case law retrieval]

## (c) Safeguards

[YOUR ASSESSMENT HERE - discuss mandatory attorney review, citation verification,
bias auditing, transparency requirements, insurance/liability]

## (d) Deployment Without Attorney Review

[YOUR ASSESSMENT HERE - take a clear position and justify it with specific
risks and benefits]
"""

print(ethical_assessment)
# Note: There is no "right" answer here. The goal is thoughtful engagement
# with the ethical dimensions of deploying AI in high-stakes domains.

**Thought Questions:**
1. If BriefEngine's retrieval disproportionately surfaces cases from eras with discriminatory legal reasoning (e.g., pre-civil-rights decisions), how could this perpetuate injustice? How would you detect and mitigate this?
2. Should an AI legal research tool tell the attorney that a particular argument it generated is "strong" or "weak"? What are the risks of the AI expressing confidence in legal conclusions?
3. BriefEngine operates under attorney-client privilege. What technical measures must be in place to ensure that one firm's case data never leaks into another firm's context?

---

## Summary

In this notebook, you built a complete context engineering pipeline for a legal research assistant. Here is what you implemented:

1. **Data Acquisition**: Loaded and chunked a corpus of legal opinions with metadata preservation
2. **Exploratory Analysis**: Analyzed the corpus structure, identifying key distributions and potential challenges
3. **Baseline**: Implemented BM25 keyword search as a performance floor
4. **Query Analysis**: Parsed attorney inputs into structured queries with jurisdiction filters (Select strategy)
5. **Filtered Retrieval**: Applied hard metadata filters to prevent irrelevant documents from entering the pipeline (Select strategy)
6. **Semantic Search + Reranking**: Used embeddings and cross-encoders for relevance ranking (Select strategy)
7. **Context Assembly**: Built a ContextEngine that allocates token budgets and structures context with XML tags (all four strategies)
8. **Citation Verification**: Created a persistent validity index to catch bad citations (Write strategy)
9. **Fine-tuning**: Trained the embedding model on legal domain data using InfoNCE contrastive loss
10. **Evaluation**: Compared the full pipeline against the baseline across multiple metrics
11. **Error Analysis**: Categorized failures into the four context failure modes
12. **Profiling**: Measured latency per pipeline stage and identified optimization opportunities
13. **Ethics**: Analyzed temporal bias and wrote an ethical impact assessment

**Key Takeaway**: The quality of an LLM's output is bounded by the quality of its context. By applying the four context engineering strategies (Write, Select, Compress, Isolate) systematically, you transformed a naive keyword-search pipeline into a precise, jurisdiction-aware, citation-verified legal research system.

**Next Steps**: Read Section 4 (Production and System Design Extension) of the case study document for a detailed look at how this pipeline would operate at scale in BriefEngine's production environment, including API design, monitoring, A/B testing, and cost analysis.

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))